In [123]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
import xgboost as xgb
import sklearn

In [124]:
from datetime import datetime, timedelta

# На вход подается массив размера (test.shape[0], 2)
def submit(preds):
    n = datetime.now() + timedelta(hours=3)
    submission = pd.read_csv('sample_submission.csv')
    #print submission.shape
    submission['TARGET'] = preds
    submission.to_csv('submission ({0}).csv'.format(n.strftime("%d-%m-%Y %H:%M")), index=0)

In [146]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

labels = train['TARGET']
train.drop(['ID'],axis=1, inplace=1)
#train.drop(['TARGET', 'ID'], axis=1, inplace=1)
test.drop(['ID'], axis=1, inplace=1)

In [147]:
unique_values = np.array([train[col].nunique() for col in train])
test.drop(train.columns[unique_values == 1], axis=1, inplace=1)
train.drop(train.columns[unique_values == 1], axis=1, inplace=1)

In [148]:
from itertools import combinations

features_duplicates = set()
visited_cols = set()

for col1 in train:
    visited_cols.add(col1)
    for col2 in train:
        if col2 in visited_cols or col2 in features_duplicates:
            continue
        if np.array_equal(train[col1], train[col2]):
            features_duplicates.add(col2)
            
print 'Дублирующихся признаков: {0}'.format(len(features_duplicates))

Дублирующихся признаков: 29


In [149]:
train.drop(features_duplicates, axis=1, inplace=1)
test.drop(features_duplicates, axis=1, inplace=1)

In [150]:
train['var38mc'] = np.isclose(train.var38, 117310.979016)
train['logvar38'] = train.loc[~train['var38mc'], 'var38'].map(np.log)
train.loc[train['var38mc'], 'logvar38'] = 0

In [151]:
test['var38mc'] = np.isclose(test.var38, 117310.979016)
test['logvar38'] = test.loc[~test['var38mc'], 'var38'].map(np.log)
test.loc[test['var38mc'], 'logvar38'] = 0

In [152]:
train.rename(columns={'var15': 'age'}, inplace=True)
test.rename(columns={'var15': 'age'}, inplace=True)

In [153]:
train = train.replace(-999999,2)
train.loc[train.var3==-999999].shape

(0, 309)

In [154]:
test = test.replace(-999999,2)
test.loc[test.var3==-999999].shape

(0, 308)

In [155]:
train['n0'] = (train==0).sum(axis=1)
test['n0'] = (test==0).sum(axis=1)

In [156]:
def xgb_predict(train,test):
    params = {}
    params['max_depth'] = 5
    #params['n_estimators']=1000
    params['learning_rate']=0.1 
    params['nthread']=4
    params['subsample']=1.0
    params['colsample_bytree']=0.5
    params['objective'] = "binary:logistic"
    params['min_child_weight'] = 3
    params['reg_alpha']=0.03
    params['seed']=1301
    params['eval_metric']= 'auc'
    params['gamma'] = 15
    params['scale_pos_weight'] = 6
    num_rounds = 100
    plst= list(params.items())
    
    
    label = train['TARGET']
    
    xgtrain = xgb.DMatrix(np.asarray(train.drop(['TARGET'],axis=1)),label)
    xgtest = xgb.DMatrix(np.asarray(test))
    
    model = xgb.train(plst,xgtrain,num_rounds)
    preds = model.predict(xgtest)
    
    return preds

In [157]:
params = {}
params['max_depth'] = 5
params['n_estimators']=100
params['learning_rate']=0.1 
params['nthread']=4
params['subsample']=1.0
params['colsample_bytree']=0.5
params['objective'] = "binary:logistic"
params['min_child_weight'] = 3
params['reg_alpha']=0.03
params['seed']=1301
params['eval_metric']= "auc"
params['gamma'] = 15
params['scale_pos_weight'] = 6
num_rounds = 100
#plst= list(params.items()) 

In [49]:
X_train, X_test, y_train, y_test = \
  cross_validation.train_test_split(train, labels, random_state=1301, stratify=labels, test_size=0.4)

In [50]:
from sklearn.metrics import roc_auc_score
preds = xgb_predict(X_train, X_test, y_train)
print roc_auc_score(y_test, preds)

0.835596892036


In [158]:
#xgall = xgb.DMatrix(train.drop(['TARGET'],axis=1), labels)

In [159]:
pr = xgb.cv(params,xgall,num_boost_round = 100, nfold = 5)

In [160]:
pr['test-auc-mean'].mean()

0.942597372

In [162]:
y_pred = xgb_predict(train,test)

In [163]:
submit(y_pred)